<a href="https://colab.research.google.com/github/KimJae-hee/solar_panel/blob/master/%ED%83%9C%EC%96%91%EA%B4%91_%EB%B0%9C%EC%A0%84_%EC%98%88%EC%B8%A1_real.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error as mse

### 우선 One-Hot Encoding으로 진행하자.

In [ ]:
sun = pd.read_csv('/content/drive/My Drive/P-SAT/태양광 예측/진짜 데이터 셋/final_data0723_3.csv', encoding = 'CP949')
sun = sun.dropna(axis = 0)
sun["ymd"] = pd.to_datetime(sun["ymd"], format = "%Y-%m-%d")
y = sun["y"]
y.index = sun["ymd"]
del sun["y"]

### 인코딩 해야 하는 변수 추출

In [ ]:
cate_list = ["month", "rain_type", "skycondition", "pm10_cat", "pm2_5_cat", "rain_yn", "hour" ]
sun_columns = sun.columns.values.tolist()

In [ ]:
non_cate_list = sun_columns
for i in range(len(cate_list)):
  non_cate_list.remove(cate_list[i])

In [ ]:
sun_cat = sun[cate_list]
sun_notcat= sun[non_cate_list]

In [ ]:
del sun_notcat["day"]
del sun_notcat["wday"]
del sun_notcat["date"]

### 범주형 변수 One-Hot Encoding

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(sun_cat)
sun_cat_ohe = enc.transform(sun_cat).toarray()

### 범주형 변수 Ordinal Encoding

In [ ]:
enc = OrdinalEncoder()
enc.fit(sun_cat)
sun_cat_ord = enc.transform(sun_cat)

### 범주형 변수 Mean Encoding

In [ ]:

def target_encoder(df, column, target, index=None, method='mean'):
    """
        df (pandas df): Pandas DataFrame containing the categorical column and target.
        column (str): Categorical variable column to be encoded.
        target (str): Target on which to encode.
        index (arr): Can be supplied to use targets only from the train index. Avoids data leakage from the test fold
        method (str): Summary statistic of the target. Mean, median or std. deviation.
    Returns:
        arr: Encoded categorical column.
    """

    index = df.index if index is None else index # Encode the entire input df if no specific indices is supplied

    if method == 'mean':
        encoded_column = df[column].map(df.iloc[index].groupby(column)[target].mean())
    elif method == 'median':
        encoded_column = df[column].map(df.iloc[index].groupby(column)[target].median())
    elif method == 'std':
        encoded_column = df[column].map(df.iloc[index].groupby(column)[target].std())
    else:
        raise ValueError("Incorrect method supplied: '{}'. Must be one of 'mean', 'median', 'std'".format(method))

    return encoded_column


### 다시 데이터 프레임 합치기

In [ ]:
x = np.array(sun_notcat)

In [ ]:
x = np.hstack((x, sun_cat_ohe))

In [ ]:
x.shape

(8665, 75)

### train, val, test set 생성

In [ ]:
x_train_1 = x[x[:,6] <= pd.to_datetime("2020-01-23", format = "%Y-%m-%d")]
x_train_2 = x[x[:,6] <= pd.to_datetime("2020-03-23", format = "%Y-%m-%d")]
x_train_3 = x[x[:,6] <= pd.to_datetime("2020-05-23", format = "%Y-%m-%d")]

x_val_1 = x[(x[:,6] >= pd.to_datetime("2020-01-24", format = "%Y-%m-%d")) & (x[:,6] <= pd.to_datetime("2020-01-30", format = "%Y-%m-%d"))]
x_val_2 = x[(x[:,6] >= pd.to_datetime("2020-03-24", format = "%Y-%m-%d")) & (x[:,6] <= pd.to_datetime("2020-03-30", format = "%Y-%m-%d"))]
x_val_3 = x[(x[:,6] >= pd.to_datetime("2020-05-24", format = "%Y-%m-%d")) & (x[:,6] <= pd.to_datetime("2020-05-30", format = "%Y-%m-%d"))]

# x_test_1 = x[x[:,6] == pd.to_datetime("2020-01-31", format = "%Y-%m-%d")]
# x_test_2 = x[x[:,6] == pd.to_datetime("2020-03-31", format = "%Y-%m-%d")]
# x_test_3 = x[x[:,6] == pd.to_datetime("2020-05-31", format = "%Y-%m-%d")]

y_train_1 = y[x[:,6] <= pd.to_datetime("2020-01-23", format = "%Y-%m-%d")]
y_train_2 = y[x[:,6] <= pd.to_datetime("2020-03-23", format = "%Y-%m-%d")]
y_train_3 = y[x[:,6] <= pd.to_datetime("2020-05-23", format = "%Y-%m-%d")]

y_val_1 = y[(x[:,6] >= pd.to_datetime("2020-01-24", format = "%Y-%m-%d")) & (x[:,6] <= pd.to_datetime("2020-01-30", format = "%Y-%m-%d"))]
y_val_2 = y[(x[:,6] >= pd.to_datetime("2020-03-24", format = "%Y-%m-%d")) & (x[:,6] <= pd.to_datetime("2020-03-30", format = "%Y-%m-%d"))]
y_val_3 = y[(x[:,6] >= pd.to_datetime("2020-05-24", format = "%Y-%m-%d")) & (x[:,6] <= pd.to_datetime("2020-05-30", format = "%Y-%m-%d"))]

# y_test_1 = y[x[:,6] == pd.to_datetime("2020-01-31", format = "%Y-%m-%d")]
# y_test_2 = y[x[:,6] == pd.to_datetime("2020-03-31", format = "%Y-%m-%d")]
# y_test_3 = y[x[:,6] == pd.to_datetime("2020-05-31", format = "%Y-%m-%d")]

sum_y_val_1 = y_val_1.groupby("ymd").sum()
sum_y_val_2 = y_val_2.groupby("ymd").sum()
sum_y_val_3 = y_val_3.groupby("ymd").sum()

TypeError: ignored

In [ ]:
def delete_timestamp(x):
  result = np.delete(x, 6, 1)
  return result
x_train_1 = delete_timestamp(x_train_1)
x_train_2 = delete_timestamp(x_train_2)
x_train_3 = delete_timestamp(x_train_3)

# x_test_1 = delete_timestamp(x_test_1)
# x_test_2 = delete_timestamp(x_test_2)
# x_test_3 = delete_timestamp(x_test_3)

x_val_1 = delete_timestamp(x_val_1)
x_val_2 = delete_timestamp(x_val_2)
x_val_3 = delete_timestamp(x_val_3)

x_train_1 = np.array(x_train_1)
x_train_2 = np.array(x_train_2)
x_train_3 = np.array(x_train_3)

x_val_1 = np.array(x_val_1)
x_val_2 = np.array(x_val_2)
x_val_3 = np.array(x_val_3)

y_val_1 = np.array(y_val_1)
y_val_2 = np.array(y_val_2)
y_val_3 = np.array(y_val_3)

y_train_1 = np.array(y_train_1)
y_train_2 = np.array(y_train_2)
y_train_3 = np.array(y_train_3)

In [ ]:
data_set_list = [[x_train_1, y_train_1, x_val_1, y_val_1],
                 [x_train_3, y_train_2, x_val_2, y_val_2],
                 [x_train_2, y_train_3, x_val_3, y_val_3]]

### 손실함수 정의

In [ ]:
def lossloss(y, y_hat):
  result = []
  for i in range(round(y.shape[0]/24)-1):
    idx = range((24*i-23), (24*i))
    tmp = sum(abs(y[idx] - yhat_val_1[idx])*y_val_1[idx])/sum(y_val_1[idx])/133
    result.append(tmp)
  loss = np.mean(result)
  return loss

def RMSE(y, y_hat):
  result = mse(y, y_hat)**0.5
  return result

### data set1(2020년 01월 31일 예측) 모델 

In [ ]:
dim_x_train_1 = x_train_2.shape
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
# 2. 모델 구성하기
model = Sequential()
model.add(Dense(1, input_dim= dim_x_train_1[1], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1))
# 3. 모델 학습과정 설정하기
model.compile(optimizer = "rmsprop", loss = root_mean_squared_error, 
              metrics =["mse"])

# 4. 모델 학습시키기
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
hist = model.fit(x_train_2, y_train_2, epochs=300, batch_size=16, validation_split = 0.2,verbose=2, callbacks=[mc, early_stopping])

# 5. 학습과정 살펴보기
import matplotlib.pyplot as plt

plt.plot(hist.history['loss'])
plt.ylim(0, 20)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

### validation 데이터 예측 및 평가

In [ ]:
yhat_val_2 = model.predict(x_val_2)
yhat_val_2 = yhat_val_2.flatten()

7.0

In [ ]:
print(RMSE(y_val_2, yhat_val_2))
print(lossloss(y_val_2, yhat_val_2))

11.503781143226856
0.24362187869337018


### 일반화 코드

### 모델 정의

In [ ]:
dim_x_train_1 = x_train_1.shape
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
# 2. 모델 구성하기
amodel = Sequential()
amodel.add(Dense(1, input_dim= dim_x_train_1[1], activation='relu'))
amodel.add(Dense(30, activation='relu'))
amodel.add(Dropout(0.3))
amodel.add(Dense(20, activation='relu'))
amodel.add(Dropout(0.3))
amodel.add(Dense(10, activation='relu'))
amodel.add(Dropout(0.3))
amodel.add(Dense(1))
# 3. 모델 학습과정 설정하기
amodel.compile(optimizer = "rmsprop", loss = root_mean_squared_error, 
              metrics =["mse"])

# 4. 모델 학습시키기
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

In [ ]:
# 데이터 세트 리스트
data_set_list = [[x_train_1, y_train_1, x_val_1, y_val_1],
                 [x_train_2, y_train_2, x_val_2, y_val_2],
                 [x_train_3, y_train_3, x_val_3, y_val_3]]

# 모델링 함수 
def modeling(train_x, train_y, val_x, val_y, amodel):
  fitting = amodel.fit(train_x, train_y, epochs = 300, batch_size = 64, validation_split = 0.3, verbose = 0, callbacks = [mc, early_stopping] )

  # validation set 예측  
  y_hat = amodel.predict(val_x)
  y_hat = y_hat.flatten()

  # 평가지표 산출 
  weighted_mean = lossloss(val_y, y_hat)
  rmse = RMSE(val_y, y_hat)

  return weighted_mean, rmse

In [ ]:
weighted = []
rmse = []
for i in range(len(data_set_list)):
  X_train = data_set_list[i][0]
  Y_train = data_set_list[i][1]
  X_val = data_set_list[i][2]
  Y_val = data_set_list[i][3]
  tmp_weighted, tmp_rmse = modeling(X_train, Y_train, X_val, Y_val, amodel)
  
  weighted.append(tmp_weighted)
  rmse.append(tmp_rmse)

  print(i, "번째 validiaion 결과는 다음과 같습니다.", ":::", "가중평균: ", tmp_weighted, "RMSE: ", tmp_rmse )

Epoch 00216: early stopping
0 번째 validiaion 결과는 다음과 같습니다. ::: 가중평균:  0.08909270690386133 RMSE:  10.423866573762693


KeyboardInterrupt: ignored

In [ ]:
print("rmse 평균은", np.mean(rmse), "\n", "가중평균은", np.mean(weighted))

rmse 평균은 15.555703874316322 
 가중평균은 0.1900217844404515


In [ ]:
for i in range(len(data_set_list)):
  X_train = data_set_list[i][0]
  Y_train = data_set_list[i][1]
  X_val = data_set_list[i][2]
  Y_val = data_set_list[i][3]
  print(X_train.shape, Y_train.shape, X_val.shape, Y_val.shape)

(4944, 57) (4944,) (168, 57) (168,)
(7800, 57) (6360,) (168, 57) (168,)
(6360, 57) (7800,) (168, 57) (168,)


# 어디선가 변수명 설정이 잘못되어 모델이 업데이트 되지 않고 있다. 그로인해 지금 결과값이 변하질 않는다... 보고 수정하자 다음에 ㅠ
